In [1]:
import pandas as pd

In [90]:
df = pd.read_csv('horse_data.csv', na_values='?')
df.columns = ['surgery', 'Age', 'Hospital Number', 'rectal_temperature', 'pulse', 'respiratory_rate', 
              'temp_of_extr', 'peripheral pulse', 'mucous membranes', 'capillary refill time', 
             'pain','peristalsis', 'abdominal distension', 'nasogastric tube', 'nasogastric reflux', 
             'nasogastric reflux PH', 'rectal examination - feces', 'abdomen', 'packed cell volume', 'total protein', 
             'abdominocentesis appearance', 'abdomcentesis total protein', 'outcome', 'surgical lesion?', 'type of lesion1', 
             'type of lesion2', 'type of lesion3', 'cp_data']

selected_df = df[['surgery', 'Age','rectal_temperature', 'pulse','respiratory_rate', 
              'temp_of_extr','pain','outcome']]
selected_df

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temp_of_extr,pain,outcome
0,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
1,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
2,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
3,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
4,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...
294,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0
295,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
296,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
297,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


In [163]:
#базовые статистики:
def check(val):
    return (val.describe())

new_df = selected_df.apply(check, axis=0)
#new_df.index = ['max_val', 'min_val', 'mean', 'mode', 'std']
new_df

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temp_of_extr,pain,outcome
count,290.000000,291.000000,291.000000,291.000000,291.000000,244.000000,244.000000,290.000000
mean,1.403448,1.604811,38.166527,71.934545,30.427386,2.348361,2.942623,1.555172
std,0.491437,2.118517,0.664499,27.878110,16.082222,1.045054,1.303993,0.743175
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.850000,48.000000,20.000000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.166527,66.000000,30.000000,3.000000,3.000000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,35.000000,3.000000,4.000000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.000000,3.000000


In [186]:
q1 = selected_df['outcome'].quantile(0.25)
q3 = selected_df['outcome'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = selected_df[selected_df['outcome'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers['outcome']

0      3.0
1      1.0
2      2.0
3      2.0
4      1.0
      ... 
294    3.0
295    3.0
296    2.0
297    1.0
298    3.0
Name: outcome, Length: 290, dtype: float64

In [187]:
pd.concat([selected_df, remove_outliers]).drop_duplicates(keep=False)
#в выбросах прошёлся по каждому столбцу. Никакого критически выбивающегося значения не нашёл

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temp_of_extr,pain,outcome
131,NaN,1,38.0,48.0,20.0,3.0,4.0,NaN


In [294]:
#Начинаем фильтровать данные
selected_df = df[['surgery', 'Age','rectal_temperature', 'pulse','respiratory_rate', 
              'temp_of_extr','pain','outcome']].dropna(thresh=4) #удаляем строки, содержащие не менее 50 %данных по столбцам
#Средняя частота дыхания здоровой лошади составляет 8-14 дыхательных движений в минуту. 
#Но в случае болезни или после тяжёлой физической работы лошадь будет дышать чаще 
#Источник: Измерение пульса и дыхания у лошади | Уход за лошадью © fourhoofs.ru == значит минимальное значение "8" в частоте дыхания - норма
#чем выше частота дыхания, тем выше пульс
selected_df.groupby('pulse')['respiratory_rate'].median()
selected_df['respiratory_rate'].fillna(selected_df.groupby('pulse')['respiratory_rate'].transform('median'), inplace=True)
selected_df.respiratory_rate.fillna(selected_df.respiratory_rate.median(),inplace=True)
selected_df.respiratory_rate.isna().sum()

0

In [218]:
#заменим в surgery пропуски модой, т.к. категориальная величина
selected_df.surgery.fillna(selected_df.surgery.mode(),inplace=True)
selected_df.loc[selected_df.surgery.isna()==True]
selected_df.surgery.isna().sum()


0

In [305]:
#судя по файлу с описанием столбцов, есть 2 категории лошадей: 
#1 = Adult horse, 2 = Young (< 6 months). Почему-то в значениях есть 9, я бы такие строки убрал, но их 22, это значительная часть данных
selected_df['Age'].value_counts() #Есть только взрослые особи. что за значени 9- неизвестно. БУДУ СЧИТАТЬ, ЧТО ЭТО МОЛОДНЯК
selected_df.Age.replace(9,2)

0      1
1      1
2      2
3      1
4      1
      ..
294    1
295    1
296    1
297    1
298    1
Name: Age, Length: 291, dtype: int64

In [302]:
selected_df.Age.replace(9,2)

0      1
1      1
2      2
3      1
4      1
      ..
294    1
295    1
296    1
297    1
298    1
Name: Age, Length: 291, dtype: int64

In [303]:
#думаю, что есть взаимосвязь между температурой и частотой дыхания.

selected_df.groupby('respiratory_rate')['rectal_temperature'].median()
selected_df['rectal_temperature'].fillna(selected_df.groupby('respiratory_rate')['rectal_temperature'].transform('median'), inplace=True)
selected_df.respiratory_rate.fillna(selected_df.respiratory_rate.median(),inplace=True)
selected_df.respiratory_rate.isna().sum()

0

In [304]:
#высокий пульс == средднее (учащенное дыхание & выше температура)

selected_df['pulse'].fillna(selected_df.groupby(['respiratory_rate', 'rectal_temperature'])['pulse'].transform('mean'), inplace=True)
selected_df.pulse.fillna(selected_df.pulse.mean(),inplace=True)
selected_df

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temp_of_extr,pain,outcome
0,1.0,1,39.20,88.0,20.0,3.0,3.0,3.0
1,2.0,1,38.30,40.0,24.0,1.0,3.0,1.0
2,1.0,9,39.10,164.0,84.0,4.0,2.0,2.0
3,2.0,1,37.30,104.0,35.0,3.0,2.0,2.0
4,2.0,1,38.45,56.0,28.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...
294,1.0,1,38.50,120.0,70.0,4.0,2.0,3.0
295,2.0,1,37.20,72.0,24.0,3.0,4.0,3.0
296,1.0,1,37.50,72.0,30.0,4.0,4.0,2.0
297,1.0,1,36.50,100.0,24.0,3.0,3.0,1.0


In [298]:
#температуру конечностей можно связать с пульсом
selected_df['temp_of_extr'].fillna(selected_df.groupby(['pulse'])['temp_of_extr'].transform('median'), inplace=True)
selected_df.temp_of_extr.fillna(selected_df.temp_of_extr.median(),inplace=True)
selected_df

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temp_of_extr,pain,outcome
0,1.0,1,39.20,88.0,20.0,3.0,3.0,3.0
1,2.0,1,38.30,40.0,24.0,1.0,3.0,1.0
2,1.0,9,39.10,164.0,84.0,4.0,2.0,2.0
3,2.0,1,37.30,104.0,35.0,3.0,NaN,2.0
4,2.0,1,38.45,56.0,28.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...
294,1.0,1,38.50,120.0,70.0,4.0,2.0,3.0
295,2.0,1,37.20,72.0,24.0,3.0,4.0,3.0
296,1.0,1,37.50,72.0,30.0,4.0,4.0,2.0
297,1.0,1,36.50,100.0,24.0,3.0,3.0,1.0


In [299]:
#заполним пропуски pain в разрезе хирургического вмешательства

selected_df['pain'].fillna(selected_df.groupby(['surgery'])['pain'].transform('median'), inplace=True)


In [284]:

selected_df['outcome'].fillna(selected_df.groupby(['surgery','pain'])['outcome'].transform('median'), inplace=True)
selected_df.outcome.fillna(selected_df.outcome.median(),inplace=True)
selected_df['outcome']

0      3.0
1      1.0
2      2.0
3      2.0
4      1.0
      ... 
294    3.0
295    3.0
296    2.0
297    1.0
298    3.0
Name: outcome, Length: 291, dtype: float64

In [300]:
selected_df

,surgery,Age,rectal_temperature,pulse,respiratory_rate,temp_of_extr,pain,outcome
0,1.0,1,39.20,88.0,20.0,3.0,3.0,3.0
1,2.0,1,38.30,40.0,24.0,1.0,3.0,1.0
2,1.0,9,39.10,164.0,84.0,4.0,2.0,2.0
3,2.0,1,37.30,104.0,35.0,3.0,2.0,2.0
4,2.0,1,38.45,56.0,28.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...
294,1.0,1,38.50,120.0,70.0,4.0,2.0,3.0
295,2.0,1,37.20,72.0,24.0,3.0,4.0,3.0
296,1.0,1,37.50,72.0,30.0,4.0,4.0,2.0
297,1.0,1,36.50,100.0,24.0,3.0,3.0,1.0
